## 4IZ460 CleverMiner Project

In [4]:
import numpy as np
import pandas as pd
import cleverminer as clm

In [5]:
df = pd.read_csv("./data/csgo_weapons.csv", sep=',')

In [6]:
df

,Name,Team,Cost,Kill Award,Kills To Rebuy,Max Speed,RoF,Damage,DPS,Armor Penetration,...,Max Ammo,Reload (CR),Reload (FR),Recoil,Recoil (V),Recoil (H),Spread,Spread Run,Spread Stand,Spread Crouch
0,AK-47,T,$2700,$300,9.00,215,600,36,360,77.50%,...,90,1.20,2.50,30.0,0,70,0.60,175.06,6.41,4.81
1,AUG,CT,$3300,$300,11.00,220,667,28,311,90.00%,...,90,1.50,3.80,26.0,0,60,0.50,135.45,3.85,2.88
2,AWP,T,$4750,$100,47.50,200,41,115,79,97.50%,...,30,2.00,3.70,78.0,15,20,0.20,176.48,80.80,60.60
3,CZ75 Auto,T,$500,$300,1.67,240,600,33,300,77.65%,...,12,0.00,2.90,27.0,10,180,3.00,13.41,10.43,7.60
4,Desert Eagle,T,$700,$300,2.33,230,267,63,280,93.20%,...,35,0.88,2.30,48.2,18,60,2.00,48.10,4.20,2.18
5,Dual Berettas,T,$500,$300,1.67,240,500,38,317,52.50%,...,120,3.00,3.82,27.0,4,20,2.00,17.85,7.00,5.25
6,FAMAS,CT,$2250,$300,7.50,220,667,30,333,70.00%,...,90,1.56,3.35,20.0,1,60,0.60,99.34,9.85,7.39
7,Five-SeveN,CT,$500,$300,1.67,240,400,32,213,91.15%,...,100,1.00,2.30,25.0,4,5,2.00,13.41,9.10,6.83
8,G3SG1,T,$5000,$300,16.67,215,240,80,320,82.50%,...,90,2.60,4.70,30.0,4,30,0.30,150.48,25.80,19.35
9,Galil AR,T,$2000,$300,6.67,215,667,30,333,77.50%,...,90,1.20,3.05,21.0,1,70,0.60,123.56,8.77,6.58
